In [1]:
%matplotlib inline

In [2]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

Helper functions to make the code more readable.



In [3]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# def prepare_sequence(seq, to_ix):
#     idxs = [to_ix[w] for w in seq]
#     return torch.tensor(idxs, dtype=torch.long)


#这里增加未登录字处理
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] if w in to_ix else to_ix["<UNK>"] for w in seq] #to_ix.has_key(x)
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

Create model



In [4]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    
    #根据转移矩阵，预测序列的得分
    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    #这个就是sentence-embedding-lstm-linear的过程，输出是linear的结果。这个输出的lstm_feats就是发射矩阵
    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    #解码，得到预测的序列，以及预测序列的得分
    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

## 自己的数据

In [5]:
f=open("ner_data.txt","r",encoding='utf8')
sentences=[]
labels=[]

In [6]:
seq_data=[]
seq_label=[]
for i in f.readlines():
    i=i.replace("\n", "")
    lst=i.split(" ")
    
    if len(lst)==2:
        seq_data.append(lst[0])
        seq_label.append(lst[1])
        
    else: #语料中是空行分隔句子
        sent=" ".join(seq_data)
        seq_data.clear()
        sentences.append(sent)
        
        label=" ".join(seq_label)
        seq_label.clear()
        labels.append(label)   

#     print(i.split(" "))

In [7]:
len(sentences),len(labels)

(50658, 50658)

In [8]:
sentences[0:5]

['当 希 望 工 程 救 助 的 百 万 儿 童 成 长 起 来 ， 科 教 兴 国 蔚 然 成 风 时 ， 今 天 有 收 藏 价 值 的 书 你 没 买 ， 明 日 就 叫 你 悔 不 当 初 ！',
 '藏 书 本 来 就 是 所 有 传 统 收 藏 门 类 中 的 第 一 大 户 ， 只 是 我 们 结 束 温 饱 的 时 间 太 短 而 已 。',
 '因 有 关 日 寇 在 京 掠 夺 文 物 详 情 ， 藏 界 较 为 重 视 ， 也 是 我 们 收 藏 北 京 史 料 中 的 要 件 之 一 。',
 '我 们 藏 有 一 册 1 9 4 5 年 6 月 油 印 的 《 北 京 文 物 保 存 保 管 状 态 之 调 查 报 告 》 ， 调 查 范 围 涉 及 故 宫 、 历 博 、 古 研 所 、 北 大 清 华 图 书 馆 、 北 图 、 日 伪 资 料 库 等 二 十 几 家 ， 言 及 文 物 二 十 万 件 以 上 ， 洋 洋 三 万 余 言 ， 是 珍 贵 的 北 京 史 料 。',
 '以 家 乡 的 历 史 文 献 、 特 定 历 史 时 期 书 刊 、 某 一 名 家 或 名 著 的 多 种 出 版 物 为 专 题 ， 注 意 精 品 、 非 卖 品 、 纪 念 品 ， 集 成 系 列 ， 那 收 藏 的 过 程 就 已 经 够 您 玩 味 无 穷 了 。']

In [9]:
labels[0:5]

['O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O',
 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O',
 'O O O B-LOC O O B-LOC O O O O O O O O O O O O O O O O O O O O B-LOC I-LOC O O O O O O O O O',
 'O O O O O O O O O O O O O O O O O B-LOC I-LOC O O O O O O O O O O O O O O O O O O O O O B-LOC I-LOC O B-LOC I-LOC O B-ORG I-ORG I-ORG O B-LOC I-LOC I-LOC I-LOC I-LOC I-LOC I-LOC O B-LOC I-LOC O B-LOC O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-LOC I-LOC O O O',
 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O']

In [10]:
data=[]

for i in range(len(sentences)):
    data.append( (sentences[i].split(), labels[i].split()) )

In [11]:
len(data)

50658

In [276]:
# training_data[0:5]
# training_data[0][0]

In [52]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {}
word_to_ix = {"<UNK>":0} #生字给id=0

for sentence, tags in data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag]=len(tag_to_ix)
        

In [53]:
len(word_to_ix),len(tag_to_ix)

(4768, 7)

In [54]:
tag_to_ix

{'O': 0,
 'B-LOC': 1,
 'I-LOC': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-PER': 5,
 'I-PER': 6}

In [55]:
tag_to_ix[START_TAG]=7
tag_to_ix[STOP_TAG]=8

In [56]:
tag_to_ix

{'O': 0,
 'B-LOC': 1,
 'I-LOC': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-PER': 5,
 'I-PER': 6,
 '<START>': 7,
 '<STOP>': 8}

In [15]:
# word_to_ix

In [44]:
from sklearn.model_selection import train_test_split

training_data,test_data=train_test_split(data,test_size=0.2, random_state=0)

In [45]:
len(training_data),len(test_data)

(40526, 10132)

### 训练

In [47]:
EMBEDDING_DIM = 50
HIDDEN_DIM = 128
EPOCHS=1

In [48]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=1e-4)

In [49]:
# Make sure prepare_sequence from earlier in the LSTM section is loaded
import time


for epoch in range(EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data
    print("epoch %d =============" % epoch)
    time_start = time.time()
    
    count=1
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix) 
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()
        
        if count%500==0:
            print("iter %d: loss %f" %(count,loss))
        count+=1
        
    time_end=time.time()
    print("time used: %d s" % (time_end-time_start)  )

epoch 0 =============
iter 500: loss 31.331604
iter 1000: loss 11.642281
iter 1500: loss 13.270813
iter 2000: loss 14.185684
iter 2500: loss 0.524353
iter 3000: loss 1.213226
iter 3500: loss 7.896988
iter 4000: loss 0.836853
iter 4500: loss 7.738739
iter 5000: loss 0.669624
iter 5500: loss 9.769913
iter 6000: loss 0.747101
iter 6500: loss 6.507309
iter 7000: loss 12.065659
iter 7500: loss 8.494522
iter 8000: loss 0.991226
iter 8500: loss 4.538750
iter 9000: loss 0.539566
iter 9500: loss 7.376556
iter 10000: loss 9.603287
iter 10500: loss 5.832260
iter 11000: loss 36.248260
iter 11500: loss 4.966980
iter 12000: loss 16.922623
iter 12500: loss 7.618118
iter 13000: loss 8.015076
iter 13500: loss 0.319382
iter 14000: loss 0.348389
iter 14500: loss 12.123367
iter 15000: loss 2.573944
iter 15500: loss 1.017059
iter 16000: loss 8.537354
iter 16500: loss 65.105072
iter 17000: loss 0.315872
iter 17500: loss 0.355728
iter 18000: loss 2.654152
iter 18500: loss 7.330177
iter 19000: loss 21.103943


### 测试

In [50]:
def compare(y, y_pred):
    error_index = []
    if len(y) == len(y_pred):
        for i in range(0, len(y)):
            if y[i] != y_pred[i]:
                error_index.append(i)

    print("error_index:",error_index)
    

In [57]:
# Check predictions before training
with torch.no_grad():  
    
    for pair in test_data[0:10]: #抽取部分看看效果
        sentence=pair[0]
        tag=pair[1]
        
        precheck_sent = prepare_sequence(sentence, word_to_ix)
        precheck_tags = torch.tensor([tag_to_ix[t] for t in tag], dtype=torch.long) 
        score,y_pred=model(precheck_sent)
            
        print(sentence)
        print(tag)
        print("实际tagid:",list(precheck_tags.numpy()))
        print("预测tagid:",y_pred)
        compare(precheck_tags,y_pred)
        
        print("====================")


['他', '说', '，', '我', '国', '正', '在', '加', '大', '治', '理', '力', '度', '，', '把', '环', '保', '工', '作', '抓', '紧', '抓', '好', '，', '到', '2', '0', '0', '0', '年', '，', '力', '争', '使', '环', '境', '污', '染', '和', '生', '态', '破', '坏', '的', '趋', '势', '得', '到', '基', '本', '控', '制', '，', '部', '分', '城', '市', '和', '地', '区', '的', '环', '境', '质', '量', '有', '所', '改', '善', '。']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
实际tagid: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
预测tagid: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### 预测

In [58]:
with torch.no_grad():  
    precheck_sent = prepare_sequence("浙江淘宝网络有限公司法定代表人发生变更，张勇卸任，由85后的蒋凡接任。", word_to_ix)
    score,y_pred=model(precheck_sent)
    print("预测tagid:",y_pred)

预测tagid: [3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [61]:
with torch.no_grad():  
    precheck_sent = prepare_sequence("他在日本帝国主义侵略者和国民党的双重压力下，威武不屈，坚持斗争。", word_to_ix)
    score,y_pred=model(precheck_sent)
    print("预测tagid:",y_pred)

预测tagid: [0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
